In [1]:
import geopandas
import scipy.io
import numpy as np
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading biological data ...


/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:274: UserWarning: We are currently using nwgt_total from Matlab for CV, change this!
  warnings.warn("We are currently using nwgt_total from Matlab for CV, change this!")


In [2]:
# get geopandas representation of final_biomass_table
df = epro_2019.final_biomass_table

krig_mesh = epro_2019.get_kriging_mesh()

# apply transformations to data using python functions
trans_df = krig_mesh.apply_longitude_transformation(df)
D_x = trans_df['Longitude'].max() - trans_df['Longitude'].min()
D_y = trans_df['Latitude'].max() - trans_df['Latitude'].min()
x, y = krig_mesh.apply_distance_transformation(trans_df, D_x, D_y)

In [3]:
# setup bins for semi-variogram calculation
nlag = 30 
lag_res = 0.002
center_bins = lag_res*np.arange(nlag)

In [4]:
# get semi-variogram class
semi_vario = epro_2019.get_semi_variogram(x, 
                                          y, df['nwgt_total'].values.flatten())

In [5]:
%%time
semi_vario.calculate_semi_variogram(center_bins)
semi_vario.gamma_standardized

CPU times: user 2.18 s, sys: 3.22 s, total: 5.4 s
Wall time: 3.86 s


array([0.55957167, 0.27871057, 0.5017043 , 0.6285182 , 0.81699518,
       0.84807799, 0.85626063, 0.88805541, 0.87678417, 0.89532938,
       0.925526  , 0.92969017, 0.92621991, 0.93359212, 0.94452939,
       0.94982919, 0.92340754, 0.91565693, 0.93720898, 0.96212605,
       0.94361212, 0.93675056, 0.96621628, 0.97465567, 0.97700998,
       0.96146162, 0.98101481, 0.98020469, 0.98974528, 0.95736852])

In [6]:
# expected output 
matlab_gamma = np.array([0.55957167, 0.27871057, 0.5017043 , 0.6285182 , 0.81699518,
       0.84807799, 0.85626063, 0.88805541, 0.87678417, 0.89532938,
       0.925526  , 0.92969017, 0.92621991, 0.93359212, 0.94452939,
       0.94982919, 0.92340754, 0.91565693, 0.93720898, 0.96212605,
       0.94361212, 0.93675056, 0.96621628, 0.97465567, 0.97700998,
       0.96146162, 0.98101481, 0.98020469, 0.98974528, 0.95736852])

np.allclose(semi_vario.gamma_standardized, matlab_gamma)

True

In [7]:
# %matplotlib widget
# semi_vario.view_semi_variogram()